# sw_audio_dsp pipeline designer

In this file you can generate the DSP pipeline of your choice.

Below you will find 4 cells which can be modified and executed to configure, tune and run the desired pipeline.



1. This is the pipeline design cell. Here you must break the DSP pipeline down into threads and use the provided DSP stages to create a pipeline. Running this cell will produce a diagram showing your pipeline. Make sure to capture each stage in your pipeline as a variable, as it will be needed in the next step.
Note that every time the pipeline cell is changed, the app must be regenerated before the tuning stage can work correctly as the stage indices used for communication may have changed.

In [ ]:
# Pipeline design stage

from audio_dsp.design.pipeline import Pipeline
from audio_dsp.stages.signal_chain import Fork, VolumeControl, Switch, Adder, Mixer, Bypass
from audio_dsp.stages.noise_suppressor_expander import NoiseSuppressorExpander
from audio_dsp.stages.cascaded_biquads import CascadedBiquads
from audio_dsp.stages.compressor_sidechain import CompressorSidechain
from audio_dsp.stages.envelope_detector import EnvelopeDetectorRMS
from audio_dsp.stages.reverb import ReverbRoom

# 6 inputs
# 0, 1: USB 
# 2, 3: live  (IN 1/2)
# 4   : Mic   (In 3)
# 5   : Mic   (In 4)
p, i = Pipeline.begin(6, fs=46000)

mus_0 = p.stage(Adder, i[0, 2])
mus_1 = p.stage(Adder, i[1, 3])
mics = p.stage(Adder, i[4:])
mic_vc = p.stage(VolumeControl, mics, label="mic_vc")
music_vc = p.stage(VolumeControl, mus_0 + mus_1, "music_vc")
peq, env_mic, peq_bypass = p.stage(Fork, mic_vc, count=3).forks
p.stage(EnvelopeDetectorRMS, env_mic, "mic_vu")

peq = p.stage(CascadedBiquads, peq, "peq")
peq_sw = p.stage(Switch, peq_bypass + peq, "peq_enable")

ns = p.stage(NoiseSuppressorExpander, peq_sw, "denoise")

p.next_thread()
rv_bypass, rv = p.stage(Fork, ns).forks
rv = p.stage(ReverbRoom, rv, "reverb", max_room_size=1.5)  # Set room size here
reverb_sw = p.stage(Switch, rv_bypass + rv, "reverb_enable")
duck_bypass, duck = p.stage(Fork, music_vc).forks

p.next_thread()
processed_mic = p.stage(Fork, reverb_sw, count=6).forks
duck0 = p.stage(CompressorSidechain, duck[0] + processed_mic[0], "duck0")
duck_sw0 = p.stage(Switch, duck_bypass[0] + duck0, "duck0_enable")
duck1 = p.stage(CompressorSidechain, duck[1] + processed_mic[1], "duck1")
duck_sw1 = p.stage(Switch, duck_bypass[1] + duck1, "duck1_enable")
loopback_0 = p.stage(Adder, duck_sw0 + processed_mic[2])
loopback_1 = p.stage(Adder, duck_sw1 + processed_mic[3])

p.next_thread()
loopback, monitor = p.stage(Fork, loopback_0 + loopback_1).forks
sw0 = p.stage(Switch, processed_mic[4] + loopback[0], "game_loopback_switch_ch0")
sw1 = p.stage(Switch, processed_mic[5] + loopback[1], "game_loopback_switch_ch1")
monitor0, monitor1 = p.stage(VolumeControl, monitor, "monitor_vc")
output_vc = p.stage(VolumeControl, sw0 + sw1, "output_vc")
monitor0, monitor_vu = p.stage(Fork, monitor0).forks
p.stage(EnvelopeDetectorRMS, monitor_vu, "out_vu")
output, live = p.stage(Fork, output_vc, count=2).forks
live = p.stage(Mixer, live)

# 0, 1: USB
# 2, 3: headphone (OUT 1/2)
# 4   : live      (OUT 3)
p.set_outputs(output + monitor0 + monitor1 + live + None)
p.draw()


2. This is the tuning cell. Parameters of each stage can be modified and these will be included in the generated DSP pipeline application.

In [ ]:
# Tuning stage

# envelope detector
p["mic_vu"].make_env_det_rms(attack_t = 0.05, release_t = 0.15)
p["out_vu"].make_env_det_rms(attack_t = 0.05, release_t = 0.15)

# PEQ
# Each entry in this peq list maps to a biquad designer method in 
# lib_audio_dsp/python/audio_dsp/dsp/biquad.py.
p["peq"].make_parametric_eq([
    ["notch", 500, 5],
    ["notch", 1000, 5],
    ["notch", 2000, 5],
    ["notch", 4000, 5],
    ["notch", 8000, 5],
    ["notch", 16000, 5],
])
p["peq"].plot_frequency_response()

# Noise suppressor, compute 2 thresholds and pasted into a header file. NS_THRESHOLD_HIGH
# and NS_THRESHOLD_LOW are toggled between when denoise is pressed
p["denoise"].make_noise_suppressor_expander(ratio=3, threshold_db=-35, attack_t=0.005, release_t=0.12)
threshold_high = p["denoise"].get_config()["threshold"]
p["denoise"].make_noise_suppressor_expander(ratio=3, threshold_db=-45, attack_t=0.005, release_t=0.12)
threshold_low = p["denoise"].get_config()["threshold"]
from pathlib import Path
Path("src/audio_dsp/dsp_pipeline/ns_thresholds.h").write_text(f"""
///AUTOGENERATED///
#pragma once
#define NS_THRESHOLD_HIGH {threshold_high}
#define NS_THRESHOLD_LOW  {threshold_low}
""")

# ducking
p["duck0"].make_compressor_sidechain(ratio=5, threshold_db=-40, attack_t=0.01, release_t=0.5)
p["duck1"].make_compressor_sidechain(ratio=5, threshold_db=-40, attack_t=0.01, release_t=0.5)


3. This is the build and run cell. This stage generates an application which uses your pipeline, and it runs the application on a connected device using xrun. The tuning parameters set in the previous cell are baked in the application.

In [ ]:
# Build and run

from audio_dsp.design.pipeline import generate_dsp_main
from build_utils import build_and_try_run

generate_dsp_main(p, out_dir="src/audio_dsp/dsp_pipeline")
build_and_try_run()
